<a href="https://colab.research.google.com/github/dubisx/EMCT_final/blob/main/Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install miditok
!pip install tokenizers
!pip install transformers

import tensorflow as tf
from tensorflow import keras
import numpy
import miditok
import tqdm
from miditok import MIDILike, MIDITokenizer, REMI
import pathlib
from miditok.constants import CHORD_MAPS
from transformers import TFGPT2LMHeadModel, GPT2Config, Trainer, TrainingArguments, GenerationConfig, pipeline

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.7/93.7 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.1/51.1 kB 2.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 38.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 89.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 25.5 MB/s eta 0:00:00


In [2]:
from google.colab import drive

drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [51]:
model_path = "/content/gdrive/MyDrive/model_f/model2/"

pitch_range = range(21, 109)
beat_res = {(0, 4): 8, (4, 12): 4}
nb_velocities = 32
additional_tokens = {'Chord': True, 'Rest': True, 'Tempo': True,
                     'rest_range': (2, 8),  # (half, 8 beats)
                     'nb_tempos': 32,  # nb of tempo bins
                     'tempo_range': (40, 250),  # (min, max)
                     'Program': False,
                     "chord_maps": CHORD_MAPS,
                     "chord_tokens_with_root_note": True,
                     "chord_unknown": False}
special_tokens = ["PAD", "BOS", "EOS"]

tokenizer = MIDILike(pitch_range, beat_res, nb_velocities, additional_tokens, special_tokens=special_tokens)

print('Loading the model...')

config = GPT2Config(
    vocab_size=1000,
    n_positions=2048,
    n_embd=512,
    n_layer=8,
    n_head=8,
    n_inner=2048,
    resid_pdrop=.1,
    embd_pdrop=.1,
    attn_pdrop=.1,
    padding_token_id=tokenizer['PAD_None'],
    bos_token_id=tokenizer['BOS_None'],
    eos_token_id=tokenizer['EOS_None'],
)
model = TFGPT2LMHeadModel(config)

#model.save_pretrained('/content/gdrive/MyDrive/models/')



Loading the model...


In [4]:
model.from_pretrained(model_path)

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at /content/gdrive/MyDrive/model_f/model2/.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [5]:

# defining our optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0)
# definining our loss function
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
# defining our metric which we want to observe
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
# compiling the model
model.compile(optimizer=optimizer, loss=[loss, *[None] * model.config.n_layer], metrics=[metric])

In [6]:
from pathlib import Path
from miditok.utils import get_midi_programs
from miditoolkit import MidiFile


midi_path = '/content/gdrive/MyDrive/example midi/midi_example.mid'

tokenizer.learn_bpe(
    vocab_size=1000,
    tokens_paths=list(Path("/content/gdrive/MyDrive/tokens_noBPE").glob('**/*.json')),
    out_dir=Path('/content/gdrive/MyDrive/tokens_BPE')
)

Loading token files: 100%|██████████| 5456/5456 [01:48<00:00, 50.42it/s]  


In [7]:
midi_path = '/content/gdrive/MyDrive/example midi/midi/midi_example.mid'

midi = MidiFile(midi_path)
token = tokenizer(midi)  # automatically detects MidiFile, paths or tokens before converting them


In [43]:
print(token[0].ids)

[328, 904, 48, 204, 52, 438, 21, 201, 226, 109, 24, 461, 112, 17, 204, 213, 702, 133, 136, 271, 38, 204, 41, 201, 45, 438, 64, 205, 217, 958, 64, 201, 65, 347, 152, 213, 64, 204, 213, 882, 105, 16, 201, 217, 587, 211, 126, 104, 679, 794, 45, 204, 48, 202, 222, 152, 214, 67, 534, 65, 338, 155, 64, 862, 133, 136, 882, 102, 129, 520, 40, 200, 43, 202, 389, 226, 128, 40, 204, 62, 202, 217, 958, 740, 201, 217, 131, 135, 958, 104, 128, 904, 48, 204, 52, 438, 21, 201, 226, 109, 24, 461, 112, 17, 204, 213, 136, 702, 133, 271, 38, 204, 41, 201, 904, 234, 105, 16, 201, 217, 126, 587, 211, 104, 679, 794, 45, 204, 48, 202, 241, 587, 820, 102, 520, 40, 200, 43, 202, 389, 226, 128, 40, 204, 225, 131, 135, 780, 104]


In [9]:
# savetok = Path('/content/gdrive/MyDrive/example midi/token1.json')

# tokenizer.save_tokens(token, savetok)

In [10]:
import json
import os

directory = '/content/gdrive/MyDrive/example midi/tokens/'


for fname in os.listdir(directory):                     # for each file in the directory
    with open(os.path.join(directory, fname)) as i:     # open the file
        data = json.load(i)   

In [49]:
prompt = [data['ids'][0][:300]]


 I have managed to convert a new MIDI to a BPE encoded token, however I am unsure on how to feed the data into the model.generate function to generate a new sequence.

In [27]:
generation_config = GenerationConfig(
    max_new_tokens=512,  # extends samples by 512 tokens
    num_beams=1,        # no beam search
    do_sample=True,     # but sample instead
    temperature=0.9,
    top_k=15,
    top_p=0.95,
    epsilon_cutoff=3e-4,
    eta_cutoff=1e-3,
    pad_token_id=config.padding_token_id,
)

In [52]:
outputs = model.generate(prompt, max_length=1000, do_sample=True, top_p=0.95, top_k=60)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [53]:
print(outputs)

tf.Tensor(
[[328 904  48 204  52 438  21 201 226 109  24 461 112  17 204 213 702 133
  136 271  38 204  41 201  45 438  64 205 217 958  64 201  65 347 152 213
   64 204 213 882 105  16 201 217 587 211 126 104 679 794  45 204  48 202
  222 152 214  67 534  65 338 155  64 862 133 136 882 102 129 520  40 200
   43 202 389 226 128  40 204  62 202 217 958 740 201 217 131 135 958 104
  128 904  48 204  52 438  21 201 226 109  24 461 112  17 204 213 136 702
  133 271  38 204  41 201 904 234 105  16 201 217 126 587 211 104 679 794
   45 204  48 202 241 587 820 102 520  40 200  43 202 389 226 128  40 204
  225 131 135 780 104 708 940 238 238 122 661  80  80 355 602 305 566  81
  870 878 747 224  44 566 800 951 305 536 744 514 708 599 837 389  65 238
  619 191 876 773 191 191 153 105 472 536 195 372 949 776  82 968 239 949
  886 744 601 489 239 744 663 864 372 536 502 647 239 705  47 744 122 601
  467 122 968  90 906 122 304 655 863 949 870 289 215 837 756 983 191 189
  154  94 211 367 467 215 2